In [2]:
import os
import urllib
import pandas as pd
import random
from statsmodels.robust import scale
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve


In [3]:
df = pd.read_csv('/Users/naina/Documents/Personal/Coding/ebay_auctions.csv')

In [18]:
df

,item,days,auctionid,bid,bidtime,bidder,bidderrate,openbid,price,delta
0,xbox,7,8211480551,52.99,1.201505,hanna1104,94.0,49.99,311.60,261.61
1,xbox,7,8211480551,50.99,1.203843,wrufai1,90.0,49.99,311.60,261.61
2,xbox,7,8211480551,101.99,1.204433,wrufai1,90.0,49.99,311.60,261.61
3,xbox,7,8211480551,57.00,1.708437,newberryhwt,14.0,49.99,311.60,261.61
4,xbox,7,8211480551,144.48,3.089711,miloo2005,3.0,49.99,311.60,261.61
5,xbox,7,8211480551,170.00,4.187963,fzakhour,5.0,49.99,311.60,261.61
6,xbox,7,8211480551,189.00,5.245810,bbcboi2007,1.0,49.99,311.60,261.61
7,xbox,7,8211480551,298.48,5.863021,miloo2005,3.0,49.99,311.60,261.61
8,xbox,7,8211480551,200.00,6.334815,laxwarrior07,12.0,49.99,311.60,261.61
9,xbox,7,8211480551,250.00,6.548796,aladdin987,292.0,49.99,311.60,261.61


In [5]:
def difference(x, y):
    return abs(x - y)

df['delta'] = difference(df['price'], df['openbid'])

In [12]:
byid = df.groupby('auctionid')

In [7]:
stats = {
    'bid': {
        'std_dev_bids': 'std',
        'average_bids': 'mean',
        'total_bids': 'count'
        },
    'delta': {
        'delta': 'mean'
    }
}

new_df = byid.agg(stats)

In [20]:
new_df

delta          bid                        
              delta average_bids total_bids std_dev_bids
auctionid                                               
1638843936  1125.00  1167.857143         14   489.252626
1638844284   300.00   362.500000          4   158.771324
1638844464   440.00   554.236250         32   130.438535
1638844729    95.00   284.545455         22    29.797295
1638893549    78.50   144.500000          5    34.022052
1638917885   202.50   135.079231         26    70.624805
1639226378  1230.00  1057.083333         24   389.372172
1639253454     5.00   252.500000          4     2.886751
1639309309   373.99   191.575556         54    86.235150
1639323228   184.00   110.187500         16    65.427791
1639333116   346.62   301.734167         48   103.775935
1639341131   251.52   127.403200         50    68.365751
1639364679   195.00  1122.000000          6    94.390678
1639425502   124.50   223.250000          4     1.443376
1639453840   354.00   149.371212         33   119.919637
1639672910   400.00  5250.000000          8   119.522861
1639826019  1125.01  1463.944444         18   336.216050
1639929579  1524.00   908.646471         34   417.558193
1639979107    98.50    58.531250         32    33.028565
1640179146   454.00   303.042727         44   129.977816
1640257270    31.46    28.069231         13     8.088999
1640495398  1300.00  1270.652174         23   412.768657
1640550476   299.00   138.000000         46   105.793510
1640653873   152.50   114.366667         30    55.462465
1640793161  2394.00   932.964286         28   806.322481
1640809330   559.00   233.445000         20   206.912246
1640809333  1724.00   862.739000         40   555.921817
1640936328   316.00   199.535000         44    98.107539
1641062012   100.01  1574.663333          6    38.479397
1641112143   380.00   758.846154         13   116.156393
...             ...          ...        ...          ...
8214735736    90.51    77.226000         10    35.822469
8214745918    83.56   151.932308         13    21.050945
8214749544   109.51    66.861111         18    30.919348
8214767887    30.01    90.892222          9    11.921806
8214772603   141.50    83.848857         35    37.356035
8214772755    79.10    70.018421         19    25.140633
8214784050    21.01    23.553333          9     8.266565
8214786481    90.01    86.312500         16    24.173591
8214801298    28.50   115.583333          6    10.170628
8214801302    39.00    21.632500         16    13.980139
8214803514    77.50    95.876154         13    28.022601
8214823984   110.00    94.600500         20    36.745589
8214862356    72.50    90.833333          9    27.970967
8214864036    67.50    85.112727         22    21.070393
8214864154    34.88    71.730000         12    12.089642
8214889177    90.00    35.824286         21    30.859769
8214917158    82.50    63.507895         19    31.281179
8214999031    92.51   118.269524         21    30.109649
8215000309   126.51    59.616000         30    42.164777
8215001975   126.51    72.432632         19    39.769424
8215125069    70.07    59.995263         19    22.142979
8215129004   121.51    45.418889         18    38.100157
8215145547    28.51   121.292857          7     6.931836
8215188105    32.51   119.244167         12    10.635298
8215408023    90.02    46.191071         28    34.979610
8215558653    80.29    92.922143         14    27.654317
8215571039   100.00   101.950000         20    31.574015
8215582227    92.50   111.131250         16    34.238413
8215605488    60.01    35.635455         11    19.136714
8215610555    30.09    22.582143         14    10.340638

[634 rows x 4 columns]

In [21]:
new_df.columns = new_df.columns.droplevel(level=0)

In [22]:
new_df

,delta,average_bids,total_bids,std_dev_bids
auctionid,,,,
1638843936,1125.00,1167.857143,14,489.252626
1638844284,300.00,362.500000,4,158.771324
1638844464,440.00,554.236250,32,130.438535
1638844729,95.00,284.545455,22,29.797295
1638893549,78.50,144.500000,5,34.022052
1638917885,202.50,135.079231,26,70.624805
1639226378,1230.00,1057.083333,24,389.372172
1639253454,5.00,252.500000,4,2.886751
1639309309,373.99,191.575556,54,86.235150


In [32]:
train, test = train_test_split(new_df, test_size = 0.5)
test

,delta,average_bids,total_bids,std_dev_bids
auctionid,,,,
1646007296,63.50,88.012500,8,27.281348
3018868918,125.72,197.676364,11,22.752285
8212752835,70.01,34.380000,25,23.125239
8212182237,62.50,95.276000,5,17.109380
3020805007,82.50,204.083333,18,20.165674
1644739924,454.00,247.736111,36,172.633767
3024361363,231.50,135.628846,26,62.255194
3018994053,177.49,103.956364,22,57.076907
8214772603,141.50,83.848857,35,37.356035


In [33]:
import statsmodels.formula.api as sm
result = sm.ols(formula="delta ~ average_bids + std_dev_bids", data=test).fit()
print(result.params)

Intercept       5.249856
average_bids    0.043416
std_dev_bids    3.005970
dtype: float64
